# Replication of the paper by Lawrence et al., 2006

https://papers.nips.cc/paper/3119-modelling-transcriptional-regulation-using-gaussian-processes.pdf

#### Probesets

The original paper restricted their interest to 5 known targets of p53: 
- DDB2 -------------- (probeset 203409_at)
- p21 ----------------- (probeset 202284_s_at) (alias p21CIP1, CDKN1A)
- SESN1/hPA26 -- (probeset 218346_s_at)
- BIK ----------------- (probeset 205780_at)
- TNFRSF10b ----- (probeset 209294_x_at, 209295_at, 210405_x_at)

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gpflow
from gpflow.utilities import print_summary, positive
plt.style.use('ggplot')
%matplotlib inline
import tensorflow as tf
import math
from tensorflow import math as tm
from sklearn import preprocessing
from scipy.interpolate import interp1d
from timeit import default_timer as timer

from tensorflow_probability import bijectors as tfb
from tensorflow_probability import distributions as tfd
from tensorflow_probability import mcmc
import tensorflow_probability as tfp

from reggae.data_loaders import load_barenco_puma, DataHolder
from reggae.gp import LinearResponseModel
from reggae.gp.options import Options

PI = tf.constant(math.pi, dtype='float64')
f64 = np.float64


In [ ]:
np.set_printoptions(threshold=np.inf)
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})


In [ ]:
m_observed, f_observed, σ2_m_pre, σ2_f_pre, t = load_barenco_puma()

m_df, m_observed = m_observed 
f_df, f_observed = f_observed
# Shape of m_observed = (replicates, genes, times)
m_observed = m_observed
f_observed = f_observed
data = (m_observed, f_observed)

σ2_m_pre = f64(σ2_m_pre)
σ2_f_pre = f64(σ2_f_pre)
noise_data = (σ2_m_pre, σ2_f_pre)

display(m_df)

num_genes = m_observed.shape[1]
N_m = m_observed.shape[2]
granularity = 100
τ = np.linspace(0, 12, granularity)
time = (t, τ, None)
data = DataHolder(data, noise_data, time)
options = Options()

model = LinearResponseModel(data, options)

print(model.Y[:8])
print(model.X.shape, model.Y.shape, model.Y_var.shape)


### Probeset Combination

TNFRSF10b has multiple probesets (probeset 209294_x_at, 209295_at, 210405_x_at) which should be combined.

It can be observed below that the log intensities have a similar pattern. Thus a popular way to combine is to take the average of the log intensities.

In [ ]:
COMBINE_MULTIPLE_PROBESETS = False
if COMBINE_MULTIPLE_PROBESETS:
    plt.figure(figsize=(10,7))
    plt.subplot(2,2,1)
    p53 = df[df.index.isin(['211300_s_at', '201746_at'])][columns].astype(float)
    for index, row in p53.iterrows():
        p53.loc[index] = np.log(list(row))
        plt.plot(list(row))

    p53_mean = pd.Series(p53.mean(0), index=genes.columns, name='p53')

    plt.subplot(2,2,2)
    TNFRSF10b = df[df.index.isin(['209294_x_at', '209295_at', '210405_x_at'])][columns]
    for index, row in TNFRSF10b.iterrows():
        print(list(row))
        TNFRSF10b.loc[index] = np.log(list(row))
        plt.plot(list(row))

    TNFRSF10b_mean = pd.Series(TNFRSF10b.mean(0), index=genes.columns, name='TNFRSF10b')
    
    plt.figure()
    plt.plot(p53_mean)

### Model

We fix the sensitivity of p21 to be 1, and decay to be 0.8 as in Barenco et al.

            K_xx = np.zeros([X.shape[0],X.shape[0]], dtype='float64')
            print(K_xx.shape)
            for j in range(num_genes):
                for k in range(num_genes):
                    K_xx[j*block_size:(j+1)*block_size, 
                         k*block_size:(k+1)*block_size] = self.k_xx(j, k, X)
            return K_xx


In [ ]:

class ExpressionKernel(gpflow.kernels.Kernel):
    def __init__(self):
        super().__init__(active_dims=[0])
        
#         l_affine = tfb.AffineScalar(shift=tf.cast(1., tf.float64),
#                             scale=tf.cast(4-1., tf.float64))
#         l_sigmoid = tfb.Sigmoid()
#         l_logistic = tfb.Chain([l_affine, l_sigmoid])

        self.lengthscale = gpflow.Parameter(1.414, transform=positive())
#         self.white_variance = gpflow.Parameter(0.1, transform=positive())

        D_affine = tfb.AffineScalar(shift=tf.cast(0.1, tf.float64),
                                    scale=tf.cast(1.5-0.1, tf.float64))
        D_sigmoid = tfb.Sigmoid()
        D_logistic = tfb.Chain([D_affine, D_sigmoid])
        S_affine = tfb.AffineScalar(shift=tf.cast(0.1, tf.float64),
                                    scale=tf.cast(4.-0.1, tf.float64))
        S_sigmoid = tfb.Sigmoid()
        S_logistic = tfb.Chain([S_affine, S_sigmoid])

        self.D = gpflow.Parameter(np.random.uniform(0.9, 1, num_genes), transform=positive(), dtype=tf.float64)
#         self.D[3].trainable = False
#         self.D[3].assign(0.8)
        self.kervar = gpflow.Parameter(np.float64(1), transform=positive())
        self.S = gpflow.Parameter(np.random.uniform(1,1, num_genes), transform=positive(), dtype=tf.float64)
#         self.S[3].trainable = False
#         self.S[3].assign(1)
        self.noise_term = gpflow.Parameter(0.1353*tf.ones(num_genes, dtype='float64'), transform=positive())
        
    def K(self, X, X2=None, calc_Kxx=False):
        self.block_size = int(X.shape[0]/num_genes)
        if calc_Kxx:
            self.hori_block_size = int(X2.shape[0])
            self.vert_block_size = int(X.shape[0]/num_genes)
            shape = [X.shape[0], X2.shape[0]*num_genes]
            K_xx = tf.zeros(shape, dtype='float64')
            for j in range(num_genes):
                for k in range(num_genes):
                    mask = np.ones(shape)
                    other = np.zeros(shape)
                    mask[j*self.vert_block_size:(j+1)*self.vert_block_size, 
                         k*self.hori_block_size:(k+1)*self.hori_block_size] = 0
                    pad_top = j*self.vert_block_size
                    pad_left = k*self.hori_block_size
                    pad_right = 0 if k == num_genes-1 else shape[1]-self.hori_block_size-pad_left
                    pad_bottom = 0 if j == num_genes-1 else shape[0]-self.vert_block_size-pad_top
                    kxx = self.k_xx(X, j, k, t_y=X2)#+self.noise_term*tf.linalg.eye(self.block_size, dtype='float64')\n",
    #                     print(kxx)
                    other = tf.pad(kxx,
                                      tf.constant([
                                          [pad_top,pad_bottom],
                                          [pad_left,pad_right]
                                   ]), 'CONSTANT'
                                  )
                    K_xx = K_xx * mask + other * (1 - mask)

    #         K_xx = self.k_xx(X, 0,0)        
#             white = tf.linalg.diag(broadcast_tile(tf.reshape(self.noise_term, (1, -1)), 1, self.block_size)[0])
            return K_xx #+ tf.linalg.diag((1e-5*tf.ones(X.shape[0], dtype='float64')))

#         tf.print(self.D, self.S)
#         interp = interp1d(np.arange(Y_var.shape[0]), Y_var.reshape(-1), kind='linear')
#         Sigma = interp(np.linspace(0,Y_var.shape[0]-1, X.shape[0]))
#         Sigma = tf.linalg.tensor_diag(Sigma)
#         White = tf.linalg.diag(
#                     tf.fill(tf.shape(X)[:-1], tf.squeeze(self.white_variance)))
        if X2 is None:
            shape = [X.shape[0],X.shape[0]]
            K_xx = tf.zeros(shape, dtype='float64')
            for j in range(num_genes):
                for k in range(num_genes):
                    mask = np.ones(shape)
                    other = np.zeros(shape)
                    mask[j*self.block_size:(j+1)*self.block_size, 
                         k*self.block_size:(k+1)*self.block_size] = 0

                    pad_top = j*self.block_size
                    pad_left = k*self.block_size
                    pad_right = 0 if k == num_genes-1 else shape[0]-self.block_size-pad_left
                    pad_bottom = 0 if j == num_genes-1 else shape[0]-self.block_size-pad_top
                    kxx = self.k_xx(X, j, k)#+self.noise_term*tf.linalg.eye(self.block_size, dtype='float64')\n",
    #                     print(kxx)
                    other = tf.pad(kxx,
                                      tf.constant([
                                          [pad_top,pad_bottom],
                                          [pad_left,pad_right]
                                   ]), 'CONSTANT'
                                  )
                    K_xx = K_xx * mask + other * (1 - mask)

    #         if X2 is None:
            shape = [X.shape[0],X.shape[0]]
    #         K_xx = self.k_xx(X, 0,0)        
            white = tf.linalg.diag(broadcast_tile(tf.reshape(self.noise_term, (1, -1)), 1, self.block_size)[0])
            return K_xx + tf.linalg.diag((1e-5*tf.ones(X.shape[0], dtype='float64'))+Y_var) + white
        else:
            '''Calculate K_xf: no need to use tf.* since this part is not optimised'''
            shape = [X.shape[0],X2.shape[0]]#self.block_size]

            K_xf = tf.zeros(shape, dtype='float64')
            for j in range(num_genes):
                mask = np.ones(shape)
                other = np.zeros(shape)
                mask[j*self.block_size:(j+1)*self.block_size] = 0
                pad_top = j*self.block_size
                pad_bottom = 0 if j == num_genes-1 else shape[0]-self.block_size-pad_top
                kxf = self.k_xf(j, X, X2)
                other = tf.pad(kxf,
                               tf.constant([[pad_top,pad_bottom],[0,0]]), 'CONSTANT'
                              )

                K_xf = K_xf * mask + other * (1 - mask)
                #[j*self.block_size:(j+1)*self.block_size] = 
            return K_xf
        
    def k_xf(self, j, X, X2):
        t_prime, t_, t_dist = self.get_distance_matrix(t_x=tf.reshape(X[:self.block_size], (-1,)), 
                                                       t_y=X2)
        l = self.lengthscale
        erf_term = tm.erf(t_dist/l - self.gamma(j)) + tm.erf(t_/l + self.gamma(j))

        return self.S[j]*l*0.5*tm.sqrt(PI)*tm.exp(self.gamma(j)**2) *tm.exp(-self.D[j]*t_dist)*erf_term 

    def _gamma(self):
        return self.D*self.lengthscale/2

    def h(self, X, k, j, t_y=None, primefirst=True):
        l = self.lengthscale
#         print(l, self.D[k], self.D[j])
        t_x = tf.reshape(X[:self.block_size], (-1,))
        t_prime, t, t_dist = self.get_distance_matrix(primefirst=primefirst, t_x=t_x, t_y=t_y)
        multiplier = tm.exp(self.gamma(k)**2) / (self.D[j]+self.D[k])
        first_erf_term = tm.erf(t_dist/l - self.gamma(k)) + tm.erf(t/l + self.gamma(k))
        second_erf_term = tm.erf(t_prime/l - self.gamma(k)) + tm.erf(self.gamma(k))
        return multiplier * (tf.multiply(tm.exp(-self.D[k]*t_dist) , first_erf_term) - \
                             tf.multiply(tm.exp(-self.D[k]*t_prime-self.D[j]*t) , second_erf_term))
    
    def gamma(self, k):
        return self.D[k]*self.lengthscale/2
    def k_xx(self, X, j, k, t_y=None):
        '''k_xx(t, tprime)'''
        mult = self.S[j]*self.S[k]*self.lengthscale*0.5*tm.sqrt(PI)
        return self.kervar**2*mult*(self.h(X, k, j, t_y=t_y) + self.h(X, j, k, t_y=t_y, primefirst=False))
    
    def h_(self, X, k, j, primefirst=True):
        Dj = tf.reshape(self.D, (1, -1))
        Dj = broadcast_tile(Dj, 1, 7)
        Dj = tf.tile(Dj, [35, 1])
        Dk = tf.reshape(self.D, (-1, 1)) 
        Dk = broadcast_tile(Dk, 7, 1)
        Dk = tf.tile(Dk, [1, 35])
        gk = tf.transpose(broadcast_tile(tf.reshape(self.gamma(), (-1, 1)), 7, 1))
        gk = tf.tile(gk, [35, 1])
        if not primefirst:
            Dk, Dj = Dj, Dk
            gk = tf.transpose(broadcast_tile(tf.reshape(self.gamma(), (1,-1)), 1, 7))
            gk = tf.tile(gk, [1, 35])

        l = self.lengthscale
        t_x = tf.reshape(X[:self.block_size], (-1,))
        t_prime, t, t_dist = self.get_distance_matrix(primefirst=primefirst, t_x=t_x)
        t_prime = tf.tile(t_prime, [5, 5])
        t = tf.tile(t, [5, 5])
        t_dist = tf.tile(t_dist, [5, 5])
        multiplier = tm.exp(gk**2) / (Dj + Dk)
        first_erf_term = tm.erf(t_dist/l - gk) + tm.erf(t/l + gk)
        second_erf_term = tm.erf(t_prime/l - gk) + tm.erf(gk)
        return multiplier * (tf.multiply(tm.exp(-Dk*t_dist) , first_erf_term) - \
                             tf.multiply(tm.exp(-Dk*t_prime-Dj*t) , second_erf_term))


    def _k_xx(self, X, j, k):
        S_square = tf.matmul(tf.reshape(self.S, (-1, 1)), tf.reshape(self.S, (1, -1)))
        S_square = broadcast_tile(S_square, 7, 7)
        mult = S_square*self.lengthscale*0.5*tm.sqrt(PI)
        return self.kervar**2*mult*(self.h(X, k, j) + self.h(X, j, k, primefirst=False))

    def get_distance_matrix(self, t_x, primefirst=True, t_y=None):
        if t_y is None:
            t_y = t_x
        t_1 = tf.transpose(tf.reshape(tf.tile(t_x, [t_y.shape[0]]), [ t_y.shape[0], t_x.shape[0]]))
        t_2 = tf.reshape(tf.tile(t_y, [t_x.shape[0]]), [ t_x.shape[0], t_y.shape[0]])
        if primefirst:
            return t_1, t_2, t_1-t_2
        return t_2, t_1, t_2-t_1
    
    def K_diag(self, X):
        print('k_diag')

        """I've used the fact that we call this method for K_ff when finding the covariance as a hack so
        I know if I should return K_ff or K_xx. In this case we're returning K_ff!!
        $K_{ff}^{post} = K_{ff} - K_{fx} K_{xx}^{-1} K_{xf}$"""
        _,_,t_dist = self.get_distance_matrix(t_x=tf.reshape(X, (-1,)))
        K_ff = tf.math.exp(-(t_dist**2)/(2*self.lengthscale**2))
        return (K_ff)


k_exp = ExpressionKernel()
print_summary(k_exp, fmt='notebook')
print(k_exp.K(X)[:7,:7])
# print(k_exp.K(X, np.linspace(0, 12, 100), calc_Kxx=True))

In [ ]:
from reggae.gp import LinearResponseMeanFunction
meanfunc_exp = LinearResponseMeanFunction(data, model.k_exp)

In [ ]:
from gpflow.logdensities import multivariate_normal
class GPR(gpflow.models.GPR):
    def log_marginal_likelihood(self) -> tf.Tensor:
        r"""
        Computes the log marginal likelihood.

        .. math::
            \log p(Y | \theta).

        """
        X, Y = self.data
        K = self.kernel(X)
        m = self.mean_function(X)
        var = tf.linalg.diag(self.likelihood.variance*tf.ones(35, dtype='float64'))
        invK = tf.linalg.inv(K+var)
        L = tf.linalg.cholesky(K+var)
        log_prob = -0.5 * tf.linalg.matmul(tf.transpose(Y-m), tf.linalg.lstsq(tf.transpose(L), tf.linalg.lstsq(L, Y-m)))#tf.matmul(tf.matmul(tf.transpose(Y-m), invK), (Y-m))
        log_prob -= 0.5 * tf.reduce_sum(tm.log(tf.linalg.diag_part(L)))
        log_prob -= np.float64(X.shape[0]/2) * tm.log(2*PI)
        return tf.reduce_sum(log_prob)

    def nll_stable(theta):
        # Numerically more stable implementation of Eq. (7) as described
        # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
        # 2.2, Algorithm 2.1.
        K = kernel(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        L = cholesky(K)
        return np.sum(np.log(np.diagonal(L))) + \
               0.5 * Y_train.T.dot(lstsq(L.T, lstsq(L, Y_train)[0])[0]) + \
               0.5 * len(X_train) * np.log(2*np.pi)

model = GPR(data=(X, Y), kernel=k_exp, mean_function=meanfunc_exp)

model.likelihood.variance.assign(3)
model.log_marginal_likelihood()

In [ ]:
gpflow.config.set_default_jitter(np.float64(1e-3))

In [ ]:
m = model.internal_model
opt = gpflow.optimizers.Scipy()
def objective_closure():
    ret = - m.log_marginal_likelihood()
    return ret


start = timer()
opt_logs = opt.minimize(objective_closure,
                        m.trainable_variables,
                        options=dict(maxiter=50, disp=True, eps=0.00000001), method='CG') # CG: 27.0
end = timer()
print(f'Time taken: {(end - start):.04f}s')

print_summary(m)


In [ ]:
np.std([55.4, 60.6])

In [ ]:
opt = gpflow.optimizers.Scipy()
opt_logs = opt.minimize(model.internal_model.training_loss, 
                        model.internal_model.trainable_variables, options=dict(maxiter=40, eps=1e-10))


In [ ]:
# See page 130 Bayesian Stats for credible intervals (not implemented here yet)

plt.figure(figsize=(15, 10))
plt.subplot(3, 3, 1)
B = m.mean_function.B.numpy()
S = m.kernel.S.numpy()#np.array([s.numpy() for s in model.kernel.S])
D = m.kernel.D.numpy()#np.array([d.numpy() for d in model.kernel.D])
B_barenco = np.array([2.6, 1.5, 0.5, 0.2, 1.35])
B_barenco = (B_barenco/np.mean(B_barenco)*np.mean(B))[[0, 4, 2, 3, 1]]
S_barenco = (np.array([3, 0.8, 0.7, 1.8, 0.7])/1.8)[[0, 4, 2, 3, 1]]
S_barenco = (S_barenco/np.mean(S_barenco)*np.mean(S))[[0, 4, 2, 3, 1]]
D_barenco = (np.array([1.2, 1.6, 1.75, 3.2, 2.3])*0.8/3.2)[[0, 4, 2, 3, 1]]
D_barenco = (D_barenco/np.mean(D_barenco)*np.mean(D))[[0, 4, 2, 3, 1]]


data = [B, S, D]
barenco_data = [B_barenco,  S_barenco, D_barenco]
vars = [0, 0,0]#[ S_mcmc, D_mcmc]
labels = ['Basal rates', 'Sensitivities', 'Decay rates']

plotnum = 331
for A, B, var, label in zip(data, barenco_data, vars, labels):
    plt.subplot(plotnum)
    plotnum+=1
    plt.bar(np.arange(5)-0.2, A, width=0.4, tick_label=m_df.index)
    plt.bar(np.arange(5)+0.2, B, width=0.4, color='blue', align='center')

    plt.title(label)
#     plt.errorbar(np.arange(5)-0.2, A, yerr=list(var.values()), fmt='none', color='green', capsize=2)
# plt.bar(range(5), S, tick_label=genes.index[:num_genes])
# plt.title('')
# plt.errorbar(range(5), S, list(S_mcmc.values()), color='green')

# plt.subplot(3, 3, 3)
# plt.bar(range(5), D, tick_label=genes.index[:num_genes])
# plt.title()

In [ ]:
granularity = 100
plt.figure(figsize=(7, 4))
pred_t = np.linspace(-2, 14, granularity, dtype='float64')
mu_post = model.predict_f(pred_t)

print(mu_post.shape)
plt.plot(pred_t, mu_post)


scale_pred = np.sqrt(np.var(mu_post));
barencof = np.array([[0.0, 200.52011, 355.5216125, 205.7574913, 135.0911372, 145.1080997, 130.7046969],
                     [0.0, 184.0994134, 308.47592, 232.1775328, 153.6595161, 85.7272235, 168.0910562],
                     [0.0, 230.2262511, 337.5994811, 276.941654, 164.5044287, 127.8653452, 173.6112139]])

print(k.K_diag(pred_t).shape)
barencof = barencof[0]/(np.sqrt(np.var(barencof[0])))*scale_pred;
lb = len(barencof)
plt.scatter(t, barencof, marker='x')

Kff = k.K_diag(pred_t)
Kff = Kff- tf.matmul(KfxKxx, Kxf)
var = tf.sqrt(tf.linalg.diag_part(Kff))
# plt.fill_between(pred_t, mu_post -var, mu_post+var)
print(Kff.shape)



In [ ]:
plt.figure(figsize=(12, 13))
pred_t = np.linspace(-2, 14, granularity, dtype='float64')
a = model.predict_x(pred_t)
for j in range(num_genes):
    plt.subplot(num_genes, 3, j+1)
    plt.plot(pred_t, a[j])
    plt.scatter(t, m_observed[0, j])
    plt.ylim(-0.2, max(a[j])*1.2)
plt.tight_layout()

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.03)

def optimization_step(model):
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(model.trainable_variables)
        loss = - model.log_marginal_likelihood()
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


for epoch in range(80):
    l = optimization_step(model)
    if epoch % 10 == 0:
        print('Epoch ', epoch, ' loss', l)
        print_summary(model)


All the Hamiltonian MCMC sampling takes place in an unconstrained space (where constrained parameters have been mapped via a bijector to an unconstrained space). This makes the optimization, as required in the gradient step, much easier.

However, we often wish to sample the constrained parameter values, not the unconstrained one. The SamplingHelper helps us convert our unconstrained values to constrained parameter ones.

In general, adaptation prevents the chain from reaching a stationary distribution, so obtaining consistent samples requires num_adaptation_steps be set to a value somewhat smaller than the number of burnin steps

step_size: Larger step sizes lead to faster progress, but too-large step sizes make rejection exponentially more likely. When possible, it's often helpful to match per-variable step sizes to the standard deviations of the target distribution in each variable

num_leapfrog_steps: Integer number of steps to run the leapfrog integrator for. Total progress per HMC step is roughly proportional to step_size * num_leapfrog_steps.


In [ ]:
get_param_dists = True

if get_param_dists:
    #https://mc-stan.org/docs/2_22/stan-users-guide/fit-gp-section.html --> priors for length-scale
    model.kernel.lengthscale.prior = tfd.Uniform(f64(3.5), f64(144))
    model.likelihood.variance.prior = tfd.Gamma(f64(1.0), f64(1.0))
    for D in model.kernel.D:
        D.prior = tfd.Gamma(f64(1), f64(1))#tfd.Normal(f64(D.numpy()), f64(0.12))
    for S in model.kernel.S:
        S.prior = tfd.Gamma(f64(1), f64(1))#tfd.Normal(f64(S.numpy()), f64(0.12))
#     for B in model.mean_function.B:
#         B.prior = tfd.Gamma(f64(1), f64(1))#tfd.Normal(f64(B.numpy()), f64(0.12))
    print_summary(model)

    num_samples = 100
    num_burnin_steps = 10

    hmc_helper = gpflow.optimizers.SamplingHelper(
        target_log_prob_fn=model.log_marginal_likelihood, 
        model_parameters = model.trainable_parameters
    )

    hmc = mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=hmc_helper.target_log_prob_fn,
        num_leapfrog_steps=5,
        step_size=0.01
    )
    adaptive_hmc = mcmc.SimpleStepSizeAdaptation(
        hmc, #mcmc.TransformedTransitionKernel(, bijectors),
        num_adaptation_steps=int(num_burnin_steps*0.8),
        target_accept_prob=f64(0.75),
        adaptation_rate=0.1
    )
#     adaptive_hmc.bootstrap_results(hmc_helper.current_state)
    
# if False:
    @tf.function
    def run_chain_fn():
#         def trace_fn(something, pkr):
#             print(something)
#             print_summary(model)
#             return pkr.inner_results.is_accepted
        return mcmc.sample_chain(
            num_results=num_samples,
            num_burnin_steps=num_burnin_steps,
            current_state=hmc_helper.current_state,
            kernel=adaptive_hmc,
            trace_fn = lambda _, pkr: pkr.inner_results.is_accepted
        )

    samples, traces = run_chain_fn()
    parameter_samples = hmc_helper.convert_constrained_values(samples)


In [ ]:
param_to_name = {param: name for name, param in
                 gpflow.utilities.parameter_dict(model).items()}

D_mcmc = {f'.kernel.D[{i}]': 0 for i in range(num_genes)}
S_mcmc = {f'.kernel.S[{i}]':0 for i in range(num_genes)}
B_mcmc = {f'.mean_function.B[{i}]':0 for i in range(num_genes)}
lengthscale = 0

plt.figure(figsize=(8,4))

for val, param in zip(parameter_samples, model.parameters):
    name = param_to_name[param]
    plt.plot(tf.squeeze(val), label=name)

    if 'lengthscale' in name:
        lengthscale = np.std(val)
    elif 'kernel.D' in name:
        D_mcmc[name] = np.std(val)
    elif 'kernel.S' in name:
        S_mcmc[name] = np.std(val)
    elif 'function.B' in name:
        B_mcmc[name] = np.std(val)

plt.legend(bbox_to_anchor=(1., 1.))
plt.xlabel('hmc iteration')
plt.ylabel('parameter_values');

print(B_mcmc, list(S_mcmc.values()))
# plot_samples(samples, 'unconstrained_variables_values')

## Non-linear response in GRN Inference notebook
    

Investigate using https://gpflow.readthedocs.io/en/master/notebooks/advanced/varying_noise.html